# Извлекаем из данных соревнования https://boosters.pro/champ_11 информацию по банкоматам и карточным терминалам

Данные могли быть заведомо зашумлены, но попытка не пытка

In [2]:
!pip install -U -q PyDrive

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


downloaded = drive.CreateFile({'id': '1Plpet7kjTUQZvdKSV9bZUE1w6yXfEuQO'})
print('Downloaded content "{}"'.format((downloaded['title'])))
downloaded.GetContentFile(downloaded['title'])

downloaded = drive.CreateFile({'id': '1mtLiIBIanehQ46CkPkf9gtFQcCGBxk2Z'})
print('Downloaded content "{}"'.format((downloaded['title'])))
downloaded.GetContentFile(downloaded['title'])

Downloaded content "test_set.csv"
Downloaded content "train_set.csv"


In [0]:
import pandas as pd
import numpy as np

In [0]:
train = pd.read_csv('train_set.csv', usecols = ['pos_adress_lat', 'pos_adress_lon', 'atm_address_lat', 'atm_address_lon','terminal_id'])
train.rename(columns = {'pos_adress_lat': 'pos_address_lat', 'pos_adress_lon': 'pos_address_lon'}, inplace = True)

test = pd.read_csv('test_set.csv', usecols = ['pos_address_lat', 'pos_address_lon', 'atm_address_lat', 'atm_address_lon'])

In [0]:
df = train.append(test)

In [67]:
df.sample(3)

,atm_address_lat,atm_address_lon,pos_address_lat,pos_address_lon,terminal_id
82387,NaN,NaN,54.516748,36.242597,NaN
1241455,NaN,NaN,NaN,NaN,NaN
743300,NaN,NaN,55.818358,49.132824,NaN


In [71]:
df_pos = df[['pos_address_lat','pos_address_lon','terminal_id']]
df_atm = df[['atm_address_lat','atm_address_lon','terminal_id']]

df_pos.rename(columns = {'pos_address_lat': 'lat', 'pos_address_lon': 'lon'}, inplace = True)
df_atm.rename(columns = {'atm_address_lat': 'lat', 'atm_address_lon': 'lon'}, inplace = True)

df_pos.dropna(inplace=True)
df_atm.dropna(inplace=True)

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3027: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [72]:
df_pos.shape, df_atm.shape

((942614, 3), (184680, 3))

In [0]:
# ограничиваем квадратом, важным для соревнования

LAT_MIN, LAT_MAX = 55.309397, 56.13526
LON_MIN, LON_MAX = 36.770379, 38.19270

In [0]:
# группируем по терминалам

df_pos = df_pos.groupby('terminal_id').mean()
df_atm = df_atm.groupby('terminal_id').mean()

In [75]:
df_pos = df_pos[df_pos['lat'] >= LAT_MIN]
df_pos = df_pos[df_pos['lat'] <= LAT_MAX]
df_pos = df_pos[df_pos['lon'] >= LON_MIN]
df_pos = df_pos[df_pos['lon'] <= LON_MAX]


df_pos.shape

(67987, 2)

In [85]:
df_pos.sample(3)

,lat,lon
terminal_id,,
ac92041c973c5c56efe30f54c774532a,55.774048,37.654861
b6b17819f36bc8e49691454c46e4ca28,56.011826,37.481120
080501321f1d3ab94c90052a1938e7dc,55.749044,37.539355


In [79]:
df_atm = df_atm[df_atm['lat'] >= LAT_MIN]
df_atm = df_atm[df_atm['lat'] <= LAT_MAX]
df_atm = df_atm[df_atm['lon'] >= LON_MIN]
df_atm = df_atm[df_atm['lon'] <= LON_MAX]


df_atm.shape

(2277, 2)

## Отобразим на карте
(чтобы не тормозило, показана 1 точка из 100, выбранная случайным образом)

In [87]:
!pip install folium

    100% |████████████████████████████████| 81kB 2.1MB/s 
  Running setup.py bdist_wheel for folium ... - \ done
  Stored in directory: /content/.cache/pip/wheels/f8/98/ff/954791afc47740d554f0d9e5885fa09dd60c2265d42578e665
Successfully built folium


In [130]:
import random

import folium # docs: http://python-visualization.github.io/folium/docs-v0.5.0/quickstart.html#Getting-Started

fmap = folium.Map([55.753722, 37.620657])

for index, row in df_atm.iterrows():
  if random.randint(1, 22) == 3:
    folium.Marker(
        location=[row.lat, row.lon],
        icon=folium.Icon(color='black'),
        popup='ATM'
    ).add_to(fmap)

for index, row in df_pos.iterrows():
  if random.randint(1, 680) == 3:
    folium.Marker(
        location=[row.lat, row.lon],
        icon=folium.Icon(color='orange'),
        popup='POS'
    ).add_to(fmap)

fmap

Итого у нас 67987 точек с терминалами и 2277 банкоматов

In [0]:
# сохраняем

df_pos.to_csv('raif_pos.csv', index=False)
df_atm.to_csv('raif_atm.csv', index=False)

from google.colab import files

files.download('raif_pos.csv')
files.download('raif_atm.csv')